#import,读取数据

In [3]:
#!/usr/bin/env python
# coding: utf-8
#导入包
import pandas as pd
import os
import gc
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

In [5]:
#数据读取
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sub = pd.DataFrame(test_df['time'])

In [3]:
#更改列名
train_df.columns = [
    'time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp',
    'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'temperature'
]
test_df.columns = [
    'time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp',
    'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo'
]

In [223]:
#train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         406 non-null    int64  
 1   year         406 non-null    int64  
 2   month        406 non-null    int64  
 3   day          406 non-null    int64  
 4   hour         406 non-null    int64  
 5   min          406 non-null    int64  
 6   sec          406 non-null    int64  
 7   outdoorTemp  404 non-null    float64
 8   outdoorHum   404 non-null    float64
 9   outdoorAtmo  404 non-null    float64
 10  indoorHum    405 non-null    float64
 11  indoorAtmo   405 non-null    float64
dtypes: float64(5), int64(7)
memory usage: 38.2 KB


#缺失值以及异常值处理

In [5]:
#第一类，上下都缺失三个
for i in range(0,train_df.shape[0]-1):
    if pd.isnull(train_df['outdoorTemp'].loc[i]) and pd.isnull(train_df['temperature'].loc[i+1]):
        train_df['outdoorTemp'].loc[i] = train_df['outdoorTemp'].loc[i+1]
        train_df['outdoorHum'].loc[i] = train_df['outdoorHum'].loc[i+1]
        train_df['outdoorAtmo'].loc[i] = train_df['outdoorAtmo'].loc[i+1]        
        train_df['indoorHum'].loc[i+1] = train_df['indoorHum'].loc[i]
        train_df['indoorAtmo'].loc[i+1] = train_df['indoorAtmo'].loc[i]
        train_df['temperature'].loc[i+1] = train_df['temperature'].loc[i]
        
#第二类，缺失六个 ，直接删除
for i in range(0,train_df.shape[0]-1):
    if pd.isnull(train_df['outdoorTemp'].loc[i]) and pd.isnull(train_df['temperature'].loc[i]):
        train_df.drop(index=i,inplace=True)
        
#剩余的缺失值 :连续缺失
train_df = train_df.fillna(method='ffill')
test_df = test_df.fillna(method='ffill')

#删除重复行
train_df.drop_duplicates(subset=['month','day','hour','min'],keep = 'first',inplace=True)

#重置索引
train_df=train_df.reset_index(drop=True)

In [225]:
#修改异常值
train_df['outdoorTemp-temperature'] = train_df['outdoorTemp'] - train_df['temperature']
train_df['diff-temperature'] = train_df['temperature'].diff(1)
train_df['diff_outdoorTemp'] = train_df['outdoorTemp'].diff(1)
train_df['diff_outdoorHum'] = train_df['outdoorHum'].diff(1)
train_df['diff_outdoorAtmo'] = train_df['outdoorAtmo'].diff(1)
train_df['diff_indoorHum'] = train_df['indoorHum'].diff(1)
train_df['diff_indoorAtmo'] = train_df['indoorAtmo'].diff(1)
test_df['diff_outdoorTemp'] = test_df['outdoorTemp'].diff(1)
test_df['diff_outdoorHum'] = test_df['outdoorHum'].diff(1)
test_df['diff_outdoorAtmo'] = test_df['outdoorAtmo'].diff(1)
test_df['diff_indoorHum'] = test_df['indoorHum'].diff(1)
test_df['diff_indoorAtmo'] = test_df['indoorAtmo'].diff(1)
for i in range(1, 24526):
    if train_df['hour'].iloc[i] != 12 and train_df['hour'].iloc[i] !=13 and train_df['hour'].iloc[i] != 1 and train_df['hour'].iloc[i] != 0:
        if (train_df['min'].iloc[i] - train_df['min'].iloc[i - 1] < 7
         and train_df['hour'].iloc[i] == train_df['hour'].iloc[i - 1])or(train_df['hour'].iloc[i] != train_df['hour'].iloc[i - 1]
         and train_df['min'].iloc[i] < 4 and train_df['min'].iloc[i - 1] > 57):
            #室外湿度
            if train_df['diff_outdoorHum'].iloc[i] > 2:
                train_df['outdoorHum'].iloc[i] = train_df['outdoorHum'].iloc[i - 1] + 0.1
            if train_df['diff_outdoorHum'].iloc[i] < -2:
                train_df['outdoorHum'].iloc[i] = train_df['outdoorHum'].iloc[i - 1] - 0.1
            #室外压强
            if train_df['diff_outdoorAtmo'].iloc[i] > 5:
                train_df['outdoorAtmo'].iloc[i] = train_df['outdoorAtmo'].iloc[i - 1] + 0.1
            if train_df['diff_outdoorAtmo'].iloc[i] < -5:
                train_df['outdoorAtmo'].iloc[i] = train_df['outdoorAtmo'].iloc[i - 1] - 0.1
            #室内湿度
            if train_df['diff_indoorHum'].iloc[i] > 2:
                train_df['indoorHum'].iloc[i] = train_df['indoorHum'].iloc[i-1] + 0.1
            if train_df['diff_indoorHum'].iloc[i] < -2:
                train_df['indoorHum'].iloc[i] = train_df['indoorHum'].iloc[i-1] - 0.1
            #室内压强
            if train_df['diff_indoorAtmo'].iloc[i] > 5:
                train_df['indoorAtmo'].iloc[i] = train_df['indoorAtmo'].iloc[i - 1] + 0.1
            if train_df['diff_indoorAtmo'].iloc[i] < -5:
                train_df['indoorAtmo'].iloc[i] = train_df['indoorAtmo'].iloc[i - 1] - 0.1
            #室外温度
            if train_df['diff_outdoorTemp'].iloc[i] >= 1:
                train_df['outdoorTemp'].iloc[i] = train_df['outdoorTemp'].iloc[i - 1] + 0.1
            if train_df['diff_outdoorTemp'].iloc[i] <= -1:
                train_df['outdoorTemp'].iloc[i] = train_df['outdoorTemp'].iloc[i - 1] - 0.1
            #室内温度
            if train_df['diff-temperature'].iloc[i] >= 1:
                train_df['temperature'].iloc[i] = train_df['temperature'].iloc[i - 1] + 0.1
            if train_df['diff-temperature'].iloc[i] <= -1:
                train_df['temperature'].iloc[i] = train_df['temperature'].iloc[i - 1] - 0.1
    elif train_df['min'].iloc[i]>=2:
        #室外湿度
        if train_df['diff_outdoorHum'].iloc[i] > 2:
            train_df['outdoorHum'].iloc[i] = train_df['outdoorHum'].iloc[i - 1] + 0.1
        if train_df['diff_outdoorHum'].iloc[i] < -2:
            train_df['outdoorHum'].iloc[i] = train_df['outdoorHum'].iloc[i - 1] - 0.1
        #室外压强
        if train_df['diff_outdoorAtmo'].iloc[i] > 5:
            train_df['outdoorAtmo'].iloc[i] = train_df['outdoorAtmo'].iloc[i - 1] + 0.1
        if train_df['diff_outdoorAtmo'].iloc[i] < -5:
            train_df['outdoorAtmo'].iloc[i] = train_df['outdoorAtmo'].iloc[i - 1] - 0.1
        #室内湿度
        if train_df['diff_indoorHum'].iloc[i] > 2:
            train_df['indoorHum'].iloc[i] = train_df['indoorHum'].iloc[i-1] + 0.1
        if train_df['diff_indoorHum'].iloc[i] < -2:
            train_df['indoorHum'].iloc[i] = train_df['indoorHum'].iloc[i-1] - 0.1
        #室内压强
        if train_df['diff_indoorAtmo'].iloc[i] > 5:
            train_df['indoorAtmo'].iloc[i] = train_df['indoorAtmo'].iloc[i - 1] + 0.1
        if train_df['diff_indoorAtmo'].iloc[i] < -5:
            train_df['indoorAtmo'].iloc[i] = train_df['indoorAtmo'].iloc[i - 1] - 0.1
        #室外温度
        if train_df['diff_outdoorTemp'].iloc[i] >= 1:
            train_df['outdoorTemp'].iloc[i] = train_df['outdoorTemp'].iloc[i - 1] + 0.1
        if train_df['diff_outdoorTemp'].iloc[i] <= -1:
            train_df['outdoorTemp'].iloc[i] = train_df['outdoorTemp'].iloc[i - 1] - 0.1
        #室内温度
        if train_df['diff-temperature'].iloc[i] >= 1:
            train_df['temperature'].iloc[i] = train_df['temperature'].iloc[i - 1] + 0.1
        if train_df['diff-temperature'].iloc[i] <= -1:
            train_df['temperature'].iloc[i] = train_df['temperature'].iloc[i - 1] - 0.1
for i in range(1, 406):
    if test_df['hour'].iloc[i] != 12 and test_df['hour'].iloc[i] !=13 and test_df['hour'].iloc[i] != 1 and test_df['hour'].iloc[i] != 0:
        if (test_df['min'].iloc[i] - test_df['min'].iloc[i - 1] < 7
         and test_df['hour'].iloc[i] == test_df['hour'].iloc[i - 1])or(test_df['hour'].iloc[i] != test_df['hour'].iloc[i - 1]
         and test_df['min'].iloc[i] < 4 and test_df['min'].iloc[i - 1] > 57):
            #室外湿度
            if test_df['diff_outdoorHum'].iloc[i] > 2:
                test_df['outdoorHum'].iloc[i] = test_df['outdoorHum'].iloc[i -1] + 0.1
            if test_df['diff_outdoorHum'].iloc[i] < -2:
                test_df['outdoorHum'].iloc[i] = test_df['outdoorHum'].iloc[i -1] - 0.1
            #室外压强
            if test_df['diff_outdoorAtmo'].iloc[i] > 5:
                test_df['outdoorAtmo'].iloc[i] = test_df['outdoorAtmo'].iloc[i - 1] + 0.1
            if test_df['diff_outdoorAtmo'].iloc[i] < -5:
                test_df['outdoorAtmo'].iloc[i] = test_df['outdoorAtmo'].iloc[i - 1] - 0.1
            #室内湿度
            if test_df['diff_indoorHum'].iloc[i] > 2:
                test_df['indoorHum'].iloc[i] = test_df['indoorHum'].iloc[i -1] + 0.1
            if test_df['diff_indoorHum'].iloc[i] < -2:
                test_df['indoorHum'].iloc[i] = test_df['indoorHum'].iloc[i - 1] - 0.1
            #室内压强
            if test_df['diff_indoorAtmo'].iloc[i] > 5:
                test_df['indoorAtmo'].iloc[i] = test_df['indoorAtmo'].iloc[i -1] + 0.1
            if test_df['diff_indoorAtmo'].iloc[i] < -5:
                test_df['indoorAtmo'].iloc[i] = test_df['indoorAtmo'].iloc[i -1] - 0.1
            #室外温度
            if test_df['diff_outdoorTemp'].iloc[i] >= 1:
                test_df['outdoorTemp'].iloc[i] = test_df['outdoorTemp'].iloc[i - 1] + 0.1
            if test_df['diff_outdoorTemp'].iloc[i] <= -1:
                test_df['outdoorTemp'].iloc[i] = test_df['outdoorTemp'].iloc[i - 1] - 0.1  
    elif test_df['min'].iloc[i]>=2 and ((test_df['min'].iloc[i] - test_df['min'].iloc[i - 1] < 7
         and test_df['hour'].iloc[i] == test_df['hour'].iloc[i - 1]) |
        (test_df['hour'].iloc[i] != test_df['hour'].iloc[i - 1]
         and test_df['min'].iloc[i] < 4 and test_df['min'].iloc[i - 1] > 57)):
        if test_df['diff_outdoorHum'].iloc[i] > 2:
            test_df['outdoorHum'].iloc[i] = test_df['outdoorHum'].iloc[i -1] + 0.1
        if test_df['diff_outdoorHum'].iloc[i] < -2:
            test_df['outdoorHum'].iloc[i] = test_df['outdoorHum'].iloc[i -1] - 0.1
        #室外压强
        if test_df['diff_outdoorAtmo'].iloc[i] > 5:
            test_df['outdoorAtmo'].iloc[i] = test_df['outdoorAtmo'].iloc[i - 1] + 0.1
        if test_df['diff_outdoorAtmo'].iloc[i] < -5:
            test_df['outdoorAtmo'].iloc[i] = test_df['outdoorAtmo'].iloc[i - 1] - 0.1
        #室内湿度
        if test_df['diff_indoorHum'].iloc[i] > 2:
            test_df['indoorHum'].iloc[i] = test_df['indoorHum'].iloc[i -1] + 0.1
        if test_df['diff_indoorHum'].iloc[i] < -2:
            test_df['indoorHum'].iloc[i] = test_df['indoorHum'].iloc[i - 1] - 0.1
        #室内压强
        if test_df['diff_indoorAtmo'].iloc[i] > 5:
            test_df['indoorAtmo'].iloc[i] = test_df['indoorAtmo'].iloc[i -1] + 0.1
        if test_df['diff_indoorAtmo'].iloc[i] < -5:
            test_df['indoorAtmo'].iloc[i] = test_df['indoorAtmo'].iloc[i -1] - 0.1
        #室外温度
        if test_df['diff_outdoorTemp'].iloc[i] >= 1:
            test_df['outdoorTemp'].iloc[i] = test_df['outdoorTemp'].iloc[i - 1] + 0.1
        if test_df['diff_outdoorTemp'].iloc[i] <= -1:
            test_df['outdoorTemp'].iloc[i] = test_df['outdoorTemp'].iloc[i - 1] - 0.1  

In [226]:
cols1=['outdoorTemp-temperature','diff-temperature','diff_outdoorTemp','diff_outdoorHum',
       'diff_outdoorAtmo','diff_indoorHum','diff_indoorAtmo']

cols2=['diff_outdoorTemp','diff_outdoorHum','diff_outdoorAtmo','diff_indoorHum','diff_indoorAtmo']
train_df.drop(cols1,axis=1,inplace=True)
test_df.drop(cols2,axis=1,inplace=True)

#滑动窗口，添加特征的统计值

In [227]:
for col in tqdm(['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']):
    train_df['avg_'+ str(2) +'_hours_' + col]=0
    train_df['max_'+ str(2) +'_hours_' + col]=0
    train_df['min_'+ str(2) +'_hours_' + col]=0
    #train_df['mad_'+ str(2) +'_hours_' + col]=0
    train_df['q1_'+ str(2) +'_hours_' + col]=0
    train_df['q3_'+ str(2) +'_hours_' + col]=0
    for i in range(0,train_df.shape[0]): #不包含当前样本的，前 2小时的 一些统计值
        if i<=60*2:
            tmp=train_df[:i][col]
        else:
            tmp=train_df[i-120:i][col]
        train_df['avg_'+ str(2) +'_hours_' + col].loc[i]=tmp.mean()
        train_df['max_'+ str(2) +'_hours_' + col].loc[i]=tmp.max()
        train_df['min_'+ str(2) +'_hours_' + col].loc[i]=tmp.min()
        #train_df['mad_'+ str(2) +'_hours_' + col].loc[i]=tmp.mad()
        train_df['q1_'+ str(2) +'_hours_' + col].loc[i]=tmp.quantile(q=0.25)
        train_df['q3_'+ str(2) +'_hours_' + col].loc[i]=tmp.quantile(q=0.75)
train_df=train_df.fillna(method='bfill')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [05:46<00:00, 69.32s/it]


In [228]:
for col in tqdm(['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']):
    test_df['avg_'+ str(2) +'_hours_' + col]=0
    test_df['max_'+ str(2) +'_hours_' + col]=0
    test_df['min_'+ str(2) +'_hours_' + col]=0
    test_df['mad_'+ str(2) +'_hours_' + col]=0
    test_df['q1_'+ str(2) +'_hours_' + col]=0
    test_df['q3_'+ str(2) +'_hours_' + col]=0
    for i in range(0,test_df.shape[0]): #不包含当前样本的，前 2小时的 一些统计值
        if i<=2*2:
            tmp=test_df[:i][col]
        else:
            tmp=test_df[i-4:i][col]
        test_df['avg_'+ str(2) +'_hours_' + col].loc[i]=tmp.mean()
        test_df['max_'+ str(2) +'_hours_' + col].loc[i]=tmp.max()
        test_df['min_'+ str(2) +'_hours_' + col].loc[i]=tmp.min()
        test_df['mad_'+ str(2) +'_hours_' + col].loc[i]=tmp.mad()
        test_df['q1_'+ str(2) +'_hours_' + col].loc[i]=tmp.quantile(q=0.25)
        test_df['q3_'+ str(2) +'_hours_' + col].loc[i]=tmp.quantile(q=0.75)
test_df=test_df.fillna(method='bfill')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.55s/it]


In [229]:
#添加了5*6=30个特征
#'avg_2_hours_outdoorTemp','avg_2_hours_outdoorHum','avg_2_hours_outdoorAtmo',
#'avg_2_hours_indoorHum','avg_2_hours_indoorAtmo'...max,mad...

#构造同期值(也可以统计多个小时之前的)，一阶差分

In [230]:
for f2 in tqdm(['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']):
    train_df['ago_1hour_'+f2]=train_df[f2].shift(1*60)
    train_df['ago_2hour_'+f2]=train_df[f2].shift(2*60)
    train_df['ago_3hour_'+f2]=train_df[f2].shift(3*60)
for f2 in tqdm(['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']):
    test_df['ago_1hour_'+f2]=test_df[f2].shift(1*2)
    test_df['ago_2hour_'+f2]=test_df[f2].shift(2*2)
    test_df['ago_3hour_'+f2]=test_df[f2].shift(3*2)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 416.91it/s]


In [231]:
#'ago_1hour_outdoorTemp','ago_1hour_outdoorHum','ago_1hour_outdoorAtmo','ago_1hour_indoorHum','ago_1hour_indoorAtmo'

此过程会产生缺失值

In [232]:
for f3 in tqdm(['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']):
    train_df['ago_1hour_'+f3+'_trend']=train_df[f3]-train_df['ago_1hour_'+f3]
    train_df['ago_2hour_'+f3+'_trend']=train_df[f3]-train_df['ago_2hour_'+f3]
    train_df['ago_3hour_'+f3+'_trend']=train_df[f3]-train_df['ago_3hour_'+f3]
for f3 in tqdm(['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']):
    test_df['ago_1hour_'+f3+'_trend']=test_df[f3]-test_df['ago_1hour_'+f3]
    test_df['ago_2hour_'+f3+'_trend']=test_df[f3]-test_df['ago_2hour_'+f3]
    test_df['ago_3hour_'+f3+'_trend']=test_df[f3]-test_df['ago_3hour_'+f3]

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 277.95it/s]


In [233]:
train_df.info()
#test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24526 entries, 0 to 24525
Data columns (total 68 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   time                         24526 non-null  int64  
 1   year                         24526 non-null  int64  
 2   month                        24526 non-null  int64  
 3   day                          24526 non-null  int64  
 4   hour                         24526 non-null  int64  
 5   min                          24526 non-null  int64  
 6   sec                          24526 non-null  int64  
 7   outdoorTemp                  24526 non-null  float64
 8   outdoorHum                   24526 non-null  float64
 9   outdoorAtmo                  24526 non-null  float64
 10  indoorHum                    24526 non-null  float64
 11  indoorAtmo                   24526 non-null  float64
 12  temperature                  24526 non-null  float64
 13  avg_2_hours_outd

#特征工程

In [234]:
train_df['ratio_outdoorTemp'] = 1 / train_df['outdoorTemp']
test_df['ratio_outdoorTemp'] = 1 / test_df['outdoorTemp']
train_df['ratio_outdoorHum'] = 1 / train_df['outdoorHum']
test_df['ratio_outdoorHum'] = 1 / test_df['outdoorHum']
train_df['ratio_outdoorAtmo'] = 1 / train_df['outdoorAtmo']
test_df['ratio_outdoorAtmo'] = 1 / test_df['outdoorAtmo']
train_df['ratio_indoorHum'] = 1 / train_df['indoorHum']
test_df['ratio_indoorHum'] = 1 / test_df['indoorHum']
train_df['ratio_indoorAtmo'] = 1 / train_df['indoorAtmo']
test_df['ratio_indoorAtmo'] = 1 / test_df['indoorAtmo']

In [ ]:
train_df['sqrt_outdoorTemp'] = np.sqrt(train_df['outdoorTemp'])
test_df['sqrt_outdoorTemp'] = np.sqrt(test_df['outdoorTemp'])
train_df['sqrt_outdoorHum'] = np.sqrt(train_df['outdoorHum'])
test_df['sqrt_outdoorHum'] = np.sqrt(test_df['outdoorHum'])
train_df['sqrt_outdoorAtmo'] = np.sqrt(train_df['outdoorAtmo'])
test_df['sqrt_outdoorAtmo'] = np.sqrt(test_df['outdoorAtmo'])
train_df['sqrt_indoorHum'] = np.sqrt(train_df['indoorHum'])
test_df['sqrt_indoorHum'] = np.sqrt(test_df['indoorHum'])
train_df['sqrt_indoorAtmo'] = np.sqrt(train_df['indoorAtmo'])
test_df['sqrt_indoorAtmo'] = np.sqrt(test_df['indoorAtmo'])

In [ ]:
#test_df['sqrt_indoorHum'].std()

In [235]:
#对训练集和测试集特征进行联合处理
data_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [236]:
# 基本聚合特征
group_feats = []
for f in tqdm(['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']):
    #基于month,day,hour做聚合特征
    data_df['MDH_{}_medi'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('median')
    data_df['MDH_{}_mean'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('mean')
    data_df['MDH_{}_max'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('max')
    data_df['MDH_{}_min'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('min')
    #data_df['MDH_{}_mad'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('mad')
    #data_df['MDH_{}_skew'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('skew')

    group_feats.append('MDH_{}_medi'.format(f))
    group_feats.append('MDH_{}_mean'.format(f))
    group_feats.append('MDH_{}_max'.format(f))
    group_feats.append('MDH_{}_min'.format(f))
    #group_feats.append('MDH_{}_mad'.format(f))
    # group_feats.append('MDH_{}_skew'.format(f))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.89it/s]


In [237]:
#group_feats = []
for f in tqdm(['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']):
    #直接基于day,hour做聚合特征
    data_df['H_{}_medi'.format(f)] = data_df.groupby(['hour'])[f].transform('median')
    data_df['H_{}_mean'.format(f)] = data_df.groupby(['hour'])[f].transform('mean')
    data_df['H_{}_max'.format(f)] = data_df.groupby(['hour'])[f].transform('max')
    data_df['H_{}_min'.format(f)] = data_df.groupby(['hour'])[f].transform('min')
    #data_df['H_{}_mad'.format(f)] = data_df.groupby(['hour'])[f].transform('mad')
    #data_df['H_{}_skew'.format(f)] = data_df.groupby(['hour'])[f].transform('skew')
    
    #group_feats.append('H_{}_medi'.format(f))
    #group_feats.append('H_{}_mean'.format(f))
    #group_feats.append('H_{}_max'.format(f))
    #group_feats.append('H_{}_min'.format(f))
    #group_feats.append('H_{}_mad'.format(f))
    #group_feats.append('H_{}_skew'.format(f))


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.37it/s]


In [ ]:
#group_feats

In [238]:
# 基本交叉特征
for f1 in tqdm(['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']+group_feats):
    for f2 in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']+group_feats:
        if f1 != f2:
            colname = '{}_{}_ratio'.format(f1, f2)
            data_df[colname] = data_df[f1].values / data_df[f2].values

data_df = data_df.fillna(method='bfill')

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 20.69it/s]


In [ ]:
data_df.shape

In [239]:
# 历史信息提取 ，提前过去5天内的数据,可以改变窗口大小
data_df['dt'] = data_df['day'].values + (data_df['month'].values - 3) * 31

for col in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        f1 = data_df[(data_df['dt']<t) & (data_df['dt']>=t-5)]
        tmp1 = f1.groupby(['hour'])[col].agg({'mean'}).reset_index()
        tmp1.columns = ['hour','hit_{}_mean'.format(col)]
        tmp1['dt'] = t
        tmp_df = tmp_df.append(tmp1)
    
    data_df = data_df.merge(tmp_df, on=['dt','hour'], how='left')
#增加了6个特征 hit_outdoorTemp_mean,hit_outdoorHum_mean...

data_df = data_df.fillna(method='bfill')    

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 30.05it/s]


In [ ]:
for col in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        f1 = data_df[(data_df['dt']<t) & (data_df['dt']>=t-5)]
        tmp1 = f1.groupby(['hour'])[col].quantile(q=0.25, axis=1).reset_index()
        tmp1.columns = ['hour','hit_{}_quantile25'.format(col)]
        tmp1['dt'] = t
        tmp_df = tmp_df.append(tmp1)
    
    data_df = data_df.merge(tmp_df, on=['dt','hour'], how='left')
    
for col in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        f1 = data_df[(data_df['dt']<t) & (data_df['dt']>=t-5)]
        tmp1 = f1.groupby(['hour'])[col].quantile(q=0.75, axis=1).reset_index()
        tmp1.columns = ['hour','hit_{}_quantile75'.format(col)]
        tmp1['dt'] = t
        tmp_df = tmp_df.append(tmp1)
    
    data_df = data_df.merge(tmp_df, on=['dt','hour'], how='left')

In [240]:
data_df.shape #增加了...个特征
#data_df.hit_outdoorTemp_mean

(24932, 724)

In [241]:
# 离散化 --分桶
#avg_2_hours_outdoorTemp
#for f1 in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
#    for f2 in ['max_2_hours_','min_2_hours_','mad_2_hours_','avg_2_hours_','q1_2_hours_','q3_2_hours_']:
#        f=f2+f1
#        data_df[f+'_20_bin'] = pd.cut(data_df[f], 20, duplicates='drop').apply(lambda x:x.left).astype(int)
#        data_df[f+'_50_bin'] = pd.cut(data_df[f], 50, duplicates='drop').apply(lambda x:x.left).astype(int)
#        data_df[f+'_100_bin'] = pd.cut(data_df[f], 100, duplicates='drop').apply(lambda x:x.left).astype(int)
#        data_df[f+'_200_bin'] = pd.cut(data_df[f], 200, duplicates='drop').apply(lambda x:x.left).astype(int)

#增加 30*4=120个特征       

for f in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
    data_df[f+'_20_bin'] = pd.cut(data_df[f], 20, duplicates='drop').apply(lambda x:x.left).astype(int)
    data_df[f+'_50_bin'] = pd.cut(data_df[f], 50, duplicates='drop').apply(lambda x:x.left).astype(int)
    data_df[f+'_100_bin'] = pd.cut(data_df[f], 100, duplicates='drop').apply(lambda x:x.left).astype(int)
    data_df[f+'_200_bin'] = pd.cut(data_df[f], 200, duplicates='drop').apply(lambda x:x.left).astype(int)
    
#分桶后 再做聚合    
for f1 in tqdm(['outdoorTemp_20_bin','outdoorHum_20_bin','outdoorAtmo_20_bin','indoorHum_20_bin','indoorAtmo_20_bin']):
    for f2 in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
        data_df['{}_{}_medi'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('min')
        #data_df['{}_{}_mad'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('mad')
       
        
for f1 in tqdm(['outdoorTemp_50_bin','outdoorHum_50_bin','outdoorAtmo_50_bin','indoorHum_50_bin','indoorAtmo_50_bin']):
    for f2 in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
        data_df['{}_{}_medi'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('min')
        #data_df['{}_{}_mad'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('mad')
        
for f1 in tqdm(['outdoorTemp_100_bin','outdoorHum_100_bin','outdoorAtmo_100_bin','indoorHum_100_bin','indoorAtmo_100_bin']):
    for f2 in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
        data_df['{}_{}_medi'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('min')
        #data_df['{}_{}_mad'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('mad')
        
for f1 in tqdm(['outdoorTemp_200_bin','outdoorHum_200_bin','outdoorAtmo_200_bin','indoorHum_200_bin','indoorAtmo_200_bin']):
    for f2 in ['outdoorTemp','outdoorHum','outdoorAtmo','indoorHum','indoorAtmo']:
        data_df['{}_{}_medi'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('min')
        #data_df['{}_{}_mad'.format(f1,f2)] = data_df.groupby([f1])[f2].transform('mad')
        

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:23<00:00,  4.72s/it]


In [242]:
data_df.shape

(24932, 1144)

#特征选择,filter ，方差选择法?

In [243]:
del_cols=[]
for c in data_df.columns:
    if data_df[c].std()<0.05:
        del_cols.append(c)
        print(c,data_df[c].std())

year 0.0
ratio_outdoorTemp 0.015279614267867284
ratio_outdoorHum 0.0046172901199715045
ratio_outdoorAtmo 8.474586787010968e-06
ratio_indoorHum 0.003982075034945189
ratio_indoorAtmo 2.203137375735311e-05
outdoorTemp_outdoorAtmo_ratio 0.004498830402544785
outdoorTemp_indoorAtmo_ratio 0.004517986210666131
outdoorTemp_MDH_outdoorTemp_medi_ratio 0.017422238077252888
outdoorTemp_MDH_outdoorTemp_mean_ratio 0.01646813938874495
outdoorTemp_MDH_outdoorTemp_max_ratio 0.02267273496778902
outdoorTemp_MDH_outdoorTemp_min_ratio 0.02539636287166447
outdoorTemp_MDH_outdoorAtmo_medi_ratio 0.004497859721774156
outdoorTemp_MDH_outdoorAtmo_mean_ratio 0.004498334814176201
outdoorTemp_MDH_outdoorAtmo_max_ratio 0.0044899958663668724
outdoorTemp_MDH_outdoorAtmo_min_ratio 0.004500045433982628
outdoorTemp_MDH_indoorAtmo_medi_ratio 0.004502412557727024
outdoorTemp_MDH_indoorAtmo_mean_ratio 0.0045024762936032225
outdoorTemp_MDH_indoorAtmo_max_ratio 0.004491440086637551
outdoorTemp_MDH_indoorAtmo_min_ratio 0.004616

In [244]:
#del_cols[1:]
del_cols=del_cols[1:]
data_df.drop(del_cols,axis=1,inplace=True)

In [245]:
data_df.shape

(24932, 825)

#分割数据集

In [246]:
#对模型进行训练并提交
drop_columns=["time","year","sec","temperature"]
train_count = train_df.shape[0]
train_df = data_df[:train_count].copy().reset_index(drop=True)
test_df = data_df[train_count:].copy().reset_index(drop=True)


features = train_df[:1].drop(drop_columns,axis=1).columns
train_x = train_df[features]
test_x = test_df[features]
test_x = test_x.fillna(method='ffill')
train_y = train_df['temperature'].values - train_df['outdoorTemp'].values


train = np.zeros((train_x.shape[0], 1))
test = np.zeros((test_x.shape[0], 1))

#LGB模型

In [247]:
nums = int(train_x.shape[0] * 0.90)
trn_x, trn_y, val_x, val_y = train_x[:nums], train_y[:nums], train_x[nums:], train_y[nums:]
    
train_matrix = lgb.Dataset(trn_x, label=trn_y)
valid_matrix = lgb.Dataset(val_x, label=val_y)
data_matrix  = lgb.Dataset(train_x, label=train_y)

In [248]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'mse',
            'min_child_weight': 5,
            'num_leaves': 32,
            'max_depth': 10,
            'objective': 'regression',
            'feature_fraction': 0.8,
            'bagging_fraction': 0.7,
            'bagging_freq': 1,
            'learning_rate': 0.005,
            'seed': 520,
            'min_data_in_leaf': 120,
            #'lambda_l1':0.01,
            #'lambda_l2':0.01
        }

In [249]:
model = lgb.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], verbose_eval=500,early_stopping_rounds=1000)
model2 = lgb.train(params, data_matrix, model.best_iteration)
val_pred = model.predict(val_x, num_iteration=model2.best_iteration).reshape(-1,1)
test_pred = model.predict(test_x, num_iteration=model2.best_iteration).reshape(-1,1)

Training until validation scores don't improve for 1000 rounds
[500]	training's l2: 0.0315756	valid_1's l2: 0.0464835
[1000]	training's l2: 0.0183116	valid_1's l2: 0.0398441
[1500]	training's l2: 0.0151275	valid_1's l2: 0.038839
[2000]	training's l2: 0.0133264	valid_1's l2: 0.0384189
[2500]	training's l2: 0.0120687	valid_1's l2: 0.0381173
[3000]	training's l2: 0.0110414	valid_1's l2: 0.037933
[3500]	training's l2: 0.0102298	valid_1's l2: 0.0378773
[4000]	training's l2: 0.00956125	valid_1's l2: 0.0378145
[4500]	training's l2: 0.00896152	valid_1's l2: 0.0377754
[5000]	training's l2: 0.00843983	valid_1's l2: 0.037762
[5500]	training's l2: 0.0079892	valid_1's l2: 0.0377651
[6000]	training's l2: 0.00758675	valid_1's l2: 0.0377407
[6500]	training's l2: 0.00722316	valid_1's l2: 0.0377804
Early stopping, best iteration is:
[5909]	training's l2: 0.00765849	valid_1's l2: 0.0376949


KeyboardInterrupt: 

In [ ]:
sub["temperature"] = test_pred[:,0] + test_df['outdoorTemp'].values
sub

In [ ]:
sub.to_csv('lgb_tc37_sub.csv', index=False)

#XGB模型

In [ ]:
nums = int(train_x.shape[0] * 0.90)
#划分测试集和验证集
trn_x, trn_y, val_x, val_y = train_x[:nums], train_y[:nums], train_x[nums:], train_y[nums:]

train_matrix = xgb.DMatrix(trn_x , label=trn_y, missing=np.nan)  #训练集
valid_matrix = xgb.DMatrix(val_x , label=val_y, missing=np.nan)  #验证集
test_matrix = xgb.DMatrix(test_x)  #测试集

In [ ]:
params = {'booster': 'gbtree',
          'eval_metric': 'rmse', 
          'min_child_weight': 5,
          'max_depth': 8,
          'subsample': 0.3,  # 减小这个参数的值，算法会更加保守，避免过拟合,典型值：0.5-1
          'colsample_bytree': 0.5,
          'eta': 0.05,
          'seed': 520,
          #'nthread': 36,
          'silent': True,
          #'alpha': 0.1    #默认1,L1正则化系数
          }

watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]  # return 训练和验证的错误率

model = xgb.train(params, train_matrix, num_boost_round=50000, evals=watchlist,
                  verbose_eval=500, early_stopping_rounds=1000)



In [ ]:
test_pred = model.predict(test_matrix , ntree_limit=model.best_ntree_limit).reshape(-1,1)

sub["temperature"] = test_pred[:,0] + test_df['outdoorTemp'].values
sub

In [ ]:
sub.to_csv('xgb_tc2_sub.csv', index=False)

#catboost模型

In [ ]:
nums = int(train_x.shape[0] * 0.90)
trn_x, trn_y, val_x, val_y = train_x[:nums], train_y[:nums], train_x[nums:], train_y[nums:]

params = {'learning_rate': 0.001,
          'depth': 5, 
          'l2_leaf_reg': 10,
          'bootstrap_type': 'Bernoulli',
          'od_type': 'Iter', 
          'od_wait': 50,
          'random_seed': 11,
          'allow_writing_files': False
         }

In [ ]:
model = CatBoostRegressor(iterations=20000, **params)
model.fit(trn_x, trn_y, eval_set=(val_x, val_y), cat_features=[], use_best_model=True, verbose=500)

val_pred  = model.predict(val_x)
test_pred = model.predict(test_x)

In [ ]:
sub["temperature"] = test_pred + test_df['outdoorTemp'].values
sub

In [ ]:
sub.to_csv('cat_tc1_sub.csv', index=False)

#Ridge模型 (交叉验证)

In [ ]:
scaler=MinMaxScaler()
train_x=scaler.fit_transform(train_x)
test_x=scaler.fit_transform(test_x)

In [ ]:
from sklearn.model_selection import GridSearchCV

#train_x,train_y,test_x 为数据
param_tc={'alpha':[0.01,0.05,0.1,0.5,1,10]}

clf=Ridge(random_state=22)
grid=GridSearchCV(estimator=clf,param_grid=param_tc,scoring='r2',cv=10) #可适当调高cv的值
#r2:决定系数，也称为拟合优度。范围[0,1],拟合优度越大，自变量对因变量的解释程度越高

grid.fit(train_x,train_y)

print(grid.best_score_)
print(grid.best_params_)

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
#选出最优参数的模型
model=Ridge(alpha=0.02,random_state=22)
model.fit(train_x,train_y)
pre=model.predict(test_x)

sub["temperature"] = pre + test_df['outdoorTemp'].values
sub

In [ ]:
sub.to_csv('Ridge_2_sub.csv', index=False)